# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('../data/sp_500_stocks.csv')
print(stocks)
type(stocks)

    Ticker
0      MMM
1      AOS
2      ABT
3     ABBV
4      ACN
..     ...
498    YUM
499   ZBRA
500    ZBH
501   ZION
502    ZTS

[503 rows x 1 columns]


pandas.core.frame.DataFrame

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
from mysecrets import IEX_CLOUD_API_TOKEN

print(IEX_CLOUD_API_TOKEN)

pk_e36ba4761f1f4d20b2a4e03ae2cfae2c


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [8]:
def makeSingleIesApiRequest(symbol):
    base_url = 'https://cloud.iexapis.com/stable/'
    suffix_url = f'?token={IEX_CLOUD_API_TOKEN}'

    api_url = base_url + f'stock/{symbol}/quote' + suffix_url
    print(api_url)

    data = requests.get(api_url)
    return data
    
response = makeSingleIesApiRequest('MMM')
print(response.status_code)
print(response.text)
dataDict = response.json()

https://cloud.iexapis.com/stable/stock/MMM/quote?token=pk_e36ba4761f1f4d20b2a4e03ae2cfae2c
200
{"avgTotalVolume":2585584,"calculationPrice":"close","change":1.32,"changePercent":0.0124,"close":107.76,"closeSource":"official","closeTime":1705698001998,"companyName":"3M Co.","currency":"USD","delayedPrice":107.76,"delayedPriceTime":1705697994618,"extendedChange":0.02,"extendedChangePercent":0.00019,"extendedPrice":107.78,"extendedPriceTime":1705712022185,"high":108.36,"highSource":"15 minute delayed price","highTime":1705697999742,"iexAskPrice":0,"iexAskSize":0,"iexBidPrice":0,"iexBidSize":0,"iexClose":107.78,"iexCloseTime":1705697996253,"iexLastUpdated":1705697996253,"iexMarketPercent":0.027303604990984845,"iexOpen":106.535,"iexOpenTime":1705674602011,"iexRealtimePrice":107.78,"iexRealtimeSize":3,"iexVolume":89269,"lastTradeTime":1705697999742,"latestPrice":107.76,"latestSource":"Close","latestTime":"January 19, 2024","latestUpdate":1705698001998,"latestVolume":3269495,"low":105.1,"lowS

In [35]:
def makeBatchIesApiRequest(symbolsString):
    base_url = 'https://cloud.iexapis.com/stable/'
    suffix_url = f'&token={IEX_CLOUD_API_TOKEN}'    #using & instead of ?, as it's already being used in this api string

    api_url = base_url + f'stock/market/batch?symbols={symbolsString}&types=quote' + suffix_url
    print(api_url)

    data = requests.get(api_url)
    return data

In [30]:
# def makeIndexApiRequest(symbol):
#     base_url = 'https://cloud.iexapis.com/stable/'
#     suffix_url = f'?token=pk_e36ba4761f1f4d20b2a4e03ae2cfae2c'

#     # api_url = base_url + f'stock/market/collection?collectionName={symbol}' + suffix_url
#     api_url = base_url + f'/stock/market/collection/sector?collectionName=nifty50' + suffix_url


#     data = requests.get(api_url)
#     return data
    
# response = makeSingleIesApiRequest('sp500')
# print(response.status_code)
# print(response.text)


https://cloud.iexapis.com/stable/stock/sp500/quote?token=pk_e36ba4761f1f4d20b2a4e03ae2cfae2c
404
Unknown symbol


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [9]:
price = dataDict['latestPrice']
marketCap = dataDict["marketCap"]
   
price, marketCap

(107.76, 59517684015)

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [10]:
myCol = ['Ticker', 'Name', 'Stock Price', 'Market Cap', 'Qty of Shares to Buy']
dataFrame = pd.DataFrame(columns=myCol)
dataFrame

,Ticker,Name,Stock Price,Market Cap,Qty of Shares to Buy


In [11]:
dataFrameNew = pd.concat(
    [
        dataFrame, 
        pd.DataFrame([['symbol', "Name", price, marketCap, 'N/A']], columns=myCol)
    ], 
    ignore_index=True
)

dataFrameNew

/var/folders/g5/jrcwvm291dd_tj25krfc65kh0000gn/T/ipykernel_13201/712535965.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataFrameNew = pd.concat(


,Ticker,Name,Stock Price,Market Cap,Qty of Shares to Buy
0,symbol,Name,107.76,59517684015,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [12]:
finalDataFrame = pd.DataFrame(columns=myCol)

for eachStock in stocks['Ticker']:
    print("for -> ", eachStock)
    stockCompleteData = makeSingleIesApiRequest(eachStock).json()

    stockRowData = pd.DataFrame([[
        eachStock, 
        stockCompleteData['companyName'],
        stockCompleteData['latestPrice'], 
        stockCompleteData['marketCap'], 
        'N/A'
    ]], columns=myCol)

    finalDataFrame = pd.concat([
        finalDataFrame, 
        stockRowData
    ])

display(finalDataFrame)


for ->  MMM
https://cloud.iexapis.com/stable/stock/MMM/quote?token=pk_e36ba4761f1f4d20b2a4e03ae2cfae2c


/var/folders/g5/jrcwvm291dd_tj25krfc65kh0000gn/T/ipykernel_13201/322874715.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finalDataFrame = pd.concat([


for ->  AOS
https://cloud.iexapis.com/stable/stock/AOS/quote?token=pk_e36ba4761f1f4d20b2a4e03ae2cfae2c
for ->  ABT
https://cloud.iexapis.com/stable/stock/ABT/quote?token=pk_e36ba4761f1f4d20b2a4e03ae2cfae2c
for ->  ABBV
https://cloud.iexapis.com/stable/stock/ABBV/quote?token=pk_e36ba4761f1f4d20b2a4e03ae2cfae2c
for ->  ACN
https://cloud.iexapis.com/stable/stock/ACN/quote?token=pk_e36ba4761f1f4d20b2a4e03ae2cfae2c
for ->  ADBE
https://cloud.iexapis.com/stable/stock/ADBE/quote?token=pk_e36ba4761f1f4d20b2a4e03ae2cfae2c
for ->  AMD
https://cloud.iexapis.com/stable/stock/AMD/quote?token=pk_e36ba4761f1f4d20b2a4e03ae2cfae2c
for ->  AES
https://cloud.iexapis.com/stable/stock/AES/quote?token=pk_e36ba4761f1f4d20b2a4e03ae2cfae2c
for ->  AFL
https://cloud.iexapis.com/stable/stock/AFL/quote?token=pk_e36ba4761f1f4d20b2a4e03ae2cfae2c
for ->  A
https://cloud.iexapis.com/stable/stock/A/quote?token=pk_e36ba4761f1f4d20b2a4e03ae2cfae2c
for ->  APD
https://cloud.iexapis.com/stable/stock/APD/quote?token=pk_e36

,Ticker,Name,Stock Price,Market Cap,Qty of Shares to Buy
0,MMM,3M Co.,107.76,59517684015,N/A
0,AOS,A.O. Smith Corp.,81.21,12406900548,N/A
0,ABT,Abbott Laboratories,114.69,199108553494,N/A
0,ABBV,Abbvie Inc,164.77,296586000000,N/A
0,ACN,Accenture plc - Class A,363.90,231074957428,N/A
...,...,...,...,...,...
0,YUM,Yum Brands Inc.,132.30,37084777374,N/A
0,ZBRA,Zebra Technologies Corp. - Class A,249.09,12793163760,N/A
0,ZBH,Zimmer Biomet Holdings Inc,123.36,25779860509,N/A
0,ZION,Zions Bancorporation N.A,41.62,6165953014,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [17]:
def chunks(stocks, n):
    for i in range(0, len(stocks), n):
        yield stocks[i:i+n]
    
symbolGroups = list(chunks(stocks, 100))
symbolGroups

[   Ticker
 0     MMM
 1     AOS
 2     ABT
 3    ABBV
 4     ACN
 ..    ...
 95     CE
 96    COR
 97    CNC
 98    CNP
 99   CDAY
 
 [100 rows x 1 columns],
     Ticker
 100     CF
 101   CHRW
 102    CRL
 103   SCHW
 104   CHTR
 ..     ...
 195    FIS
 196   FITB
 197   FSLR
 198     FE
 199     FI
 
 [100 rows x 1 columns],
     Ticker
 200    FLT
 201    FMC
 202      F
 203   FTNT
 204    FTV
 ..     ...
 295   LULU
 296    LYB
 297    MTB
 298    MRO
 299    MPC
 
 [100 rows x 1 columns],
     Ticker
 300   MKTX
 301    MAR
 302    MMC
 303    MLM
 304    MAS
 ..     ...
 395      O
 396    REG
 397   REGN
 398     RF
 399    RSG
 
 [100 rows x 1 columns],
     Ticker
 400    RMD
 401   RVTY
 402    RHI
 403    ROK
 404    ROL
 ..     ...
 495   WYNN
 496    XEL
 497    XYL
 498    YUM
 499   ZBRA
 
 [100 rows x 1 columns],
     Ticker
 500    ZBH
 501   ZION
 502    ZTS]

In [26]:
symbolGroupsStringList = []

for group in symbolGroups:
    symbolGroupsString = ','.join(group['Ticker'])    
    symbolGroupsStringList.append(symbolGroupsString)

print(symbolGroupsStringList)
len(symbolGroupsString)

['MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,APD,ABNB,AKAM,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,AME,AMGN,APH,ADI,ANSS,AON,APA,AAPL,AMAT,APTV,ACGL,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BK,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BX,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,BLDR,BG,CDNS,CZR,CPT,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,COR,CNC,CNP,CDAY', 'CF,CHRW,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CTVA,CSGP,COST,CTRA,CCI,CSX,CMI,CVS,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DG,DLTR,D,DPZ,DOV,DOW,DHI,DTE,DUK,DD,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,LLY,EMR,ENPH,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ESS,EL,ETSY,EG,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FIS,FITB,FSLR,FE,FI', 'FLT,FMC,F,FTNT,FTV,FOXA,FOX,BEN,FCX,GRMN,IT,GEHC,GEN,GNRC,GD,GE,GIS,GM,GPC,GILD,GPN,GL,GS,HAL,HIG,HAS,HCA,PEAK,HSIC,HSY,HES,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,

6

In [43]:
finalDataFrame = pd.DataFrame(columns=myCol)

for eachStringGroup in symbolGroupsStringList: 
    response = makeBatchIesApiRequest(eachStringGroup)
    print(response.status_code)

    responseDict = response.json()
    
    for eachStock in responseDict:
        # print(eachStock)

        stockCompleteData = responseDict[eachStock]['quote']

        stockRowData = pd.DataFrame([[
            eachStock, 
            stockCompleteData['companyName'],
            stockCompleteData['latestPrice'], 
            stockCompleteData['marketCap'], 
            'N/A'
        ]], columns=myCol)
        
        finalDataFrame = pd.concat([
            finalDataFrame,
            stockRowData
        ], ignore_index=True)

finalDataFrame

https://cloud.iexapis.com/stable/stock/market/batch?symbols=MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,APD,ABNB,AKAM,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,AME,AMGN,APH,ADI,ANSS,AON,APA,AAPL,AMAT,APTV,ACGL,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BK,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BX,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,BLDR,BG,CDNS,CZR,CPT,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,COR,CNC,CNP,CDAY&types=quote&token=pk_e36ba4761f1f4d20b2a4e03ae2cfae2c
200
https://cloud.iexapis.com/stable/stock/market/batch?symbols=CF,CHRW,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CTVA,CSGP,COST,CTRA,CCI,CSX,CMI,CVS,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DG,DLTR,D,DPZ,DOV,DOW,DHI,DTE,DUK,DD,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,LLY,EMR,ENPH,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ESS,EL,ETSY,EG,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FIS,FI

/var/folders/g5/jrcwvm291dd_tj25krfc65kh0000gn/T/ipykernel_13201/918168820.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finalDataFrame = pd.concat([


200
https://cloud.iexapis.com/stable/stock/market/batch?symbols=FLT,FMC,F,FTNT,FTV,FOXA,FOX,BEN,FCX,GRMN,IT,GEHC,GEN,GNRC,GD,GE,GIS,GM,GPC,GILD,GPN,GL,GS,HAL,HIG,HAS,HCA,PEAK,HSIC,HSY,HES,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HUBB,HUM,HBAN,HII,IBM,IEX,IDXX,ITW,ILMN,INCY,IR,PODD,INTC,ICE,IFF,IP,IPG,INTU,ISRG,IVZ,INVH,IQV,IRM,JBHT,JBL,JKHY,J,JNJ,JCI,JPM,JNPR,K,KVUE,KDP,KEY,KEYS,KMB,KIM,KMI,KLAC,KHC,KR,LHX,LH,LRCX,LW,LVS,LDOS,LEN,LIN,LYV,LKQ,LMT,L,LOW,LULU,LYB,MTB,MRO,MPC&types=quote&token=pk_e36ba4761f1f4d20b2a4e03ae2cfae2c
200
https://cloud.iexapis.com/stable/stock/market/batch?symbols=MKTX,MAR,MMC,MLM,MAS,MA,MTCH,MKC,MCD,MCK,MDT,MRK,META,MET,MTD,MGM,MCHP,MU,MSFT,MAA,MRNA,MHK,MOH,TAP,MDLZ,MPWR,MNST,MCO,MS,MOS,MSI,MSCI,NDAQ,NTAP,NFLX,NEM,NWSA,NWS,NEE,NKE,NI,NDSN,NSC,NTRS,NOC,NCLH,NRG,NUE,NVDA,NVR,NXPI,ORLY,OXY,ODFL,OMC,ON,OKE,ORCL,OTIS,PCAR,PKG,PANW,PARA,PH,PAYX,PAYC,PYPL,PNR,PEP,PFE,PCG,PM,PSX,PNW,PXD,PNC,POOL,PPG,PPL,PFG,PG,PGR,PLD,PRU,PEG,PTC,PSA,PHM,QRVO,PWR,QCOM,DGX,RL,RJF,RTX,O,REG,R

,Ticker,Name,Stock Price,Market Cap,Qty of Shares to Buy
0,MMM,3M Co.,107.76,59517684015,N/A
1,AOS,A.O. Smith Corp.,81.21,12406900548,N/A
2,ABT,Abbott Laboratories,114.69,199108553494,N/A
3,ABBV,Abbvie Inc,164.77,296586000000,N/A
4,ACN,Accenture plc - Class A,363.90,231074957428,N/A
...,...,...,...,...,...
498,YUM,Yum Brands Inc.,132.30,37084777374,N/A
499,ZBRA,Zebra Technologies Corp. - Class A,249.09,12793163760,N/A
500,ZBH,Zimmer Biomet Holdings Inc,123.36,25779860509,N/A
501,ZION,Zions Bancorporation N.A,41.62,6165953014,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: